In [1]:
from tensorflow import keras
from keras.layers import Layer
import keras.backend as K
import tensorflow as tf


from keras.datasets import cifar10
from keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPool2D,  \
    Dropout, Dense, Input, concatenate,      \
    GlobalAveragePooling2D, AveragePooling2D,\
    Flatten, BatchNormalization, Activation, MaxPooling2D, GlobalMaxPooling2D,\
    Reshape,  multiply, add, Permute

import math
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler
from keras.regularizers import l2

2024-03-18 07:13:17.104601: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-18 07:13:17.104690: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-18 07:13:17.104721: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-18 07:13:17.117042: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import tensorflow as tf
import imageio
import numpy as np
import os


def read_images(dir):

    supported_extensions = [".jpg", ".jpeg", ".png"]
    image_list = []
    count = 0
    # Walk through the directory and read images
    for root, _, files in os.walk(dir):
        for file in files:
            file_extension = os.path.splitext(file)[-1].lower()

            # Check if the file is a .jpg or .jpeg image
            if file_extension in supported_extensions:
                image_path = os.path.join(root, file)
                try:
                    image = imageio.imread(image_path)
                    image1 = image

                    image = np.asarray(image)
                    del image1
                    image_list.append(image)
                except Exception as e:
                    print(f"Error reading image {image_path}: {e}")

            count += 1

            if count % 100 == 0:
              print(str(count) + " images read")


    return image_list



In [3]:
# Train data directories
base_train_dir = '../workspace/Banana2/V2/Test/'

# Reading images for new classes
cordana = read_images(os.path.join(base_train_dir, 'cordana'))
healthy = read_images(os.path.join(base_train_dir, 'healthy'))  # This class remains common
pestalotiopsis = read_images(os.path.join(base_train_dir, 'pestalotiopsis'))
sigatoka = read_images(os.path.join(base_train_dir, 'sigatoka'))

# Labeling for new train classes
labels_cordana = np.zeros(len(cordana))
labels_pestalotiopsis = np.ones(len(pestalotiopsis))
# Healthy remains common, updating its label to reflect new class ordering
labels_healthy = np.full(len(healthy), 2)
labels_sigatoka = np.full(len(sigatoka), 3)

# Combine train labels for new class setup
labels = np.concatenate([
    labels_cordana,
    labels_pestalotiopsis,
    labels_healthy,
    labels_sigatoka
])

print(len(cordana) + len(pestalotiopsis) + len(healthy) + len(sigatoka))

# Relative Frequencies for new classes
# Create an array to store the frequencies of the lists
class_freq = []

# Append the lengths of the lists to the array for new classes
class_freq.append(len(cordana))
class_freq.append(len(pestalotiopsis))
class_freq.append(len(healthy))
class_freq.append(len(sigatoka))

# Calculate the sum of all frequencies
total_freq = sum(class_freq)

# Calculate the relative frequencies for new classes
relative_freq = [freq / total_freq for freq in class_freq]

# Combine all images into a single list for processing or analysis
images = cordana + pestalotiopsis + healthy + sigatoka


/tmp/ipykernel_96387/1347079695.py:21: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(image_path)


100 images read
200 images read
100 images read
200 images read
100 images read
100 images read
200 images read


840

In [8]:
len(images)

840

In [9]:
tf.__version__

'2.14.0'

In [10]:
# !pip install tensorflow==2.13

In [11]:
# model = keras.models.load_model('drive/MyDrive/C2_Models/best_epoch33_CustomModel_Full_part2 (7).keras')
model = keras.models.load_model('../workspace/MobileNet/epoch15_model_part1.keras')

2024-03-18 07:13:21.766007: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 79086 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:00:09.0, compute capability: 8.0
2024-03-18 07:13:23.588029: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


In [12]:
import imgaug as ia
import imgaug.augmenters as iaa
import random
import copy

# Find accuracy of model
def find_accuracy(test,pred):
    correct = 0
    total = len(test)

    for i in range(len(test)):
        if test[i] == pred[i]:
            correct += 1

    return correct/total


# Map ANN outputs to classes
def get_labels(y_pred_ann):
    labels = []

    for pred in y_pred_ann:
        max_index = 0

        for i in range(len(pred)):
            if pred[i] > pred[max_index]:
                max_index = i

        labels.append(max_index)

    return labels


def resize_images(images_list, width=128, height=128):
    ia.seed(1)
    # Define the resize augmentation
    resize_augmenter = iaa.Resize({"height": height, "width": width})

    resized_images = []

    for image in images_list:
        # Ensure the image is in RGB format (imgaug uses RGB by default)
        if image.shape[-1] == 1:  # Grayscale image with single channel
            image = np.repeat(image, 3, axis=-1)

        # Apply the resize augmentation
        augmented_image = resize_augmenter.augment_image(image)

        # Append the augmented image to the result list
        resized_images.append(augmented_image)

    del images_list[:]
    return resized_images

def normalize_images(image_list):
  for i in range(len(image_list)):
    image = image_list[i].astype(np.float32) / 255.0
    image_list[i] = image

In [13]:

images = resize_images(images, 224, 224)
normalize_images(images)
images = np.array(images)

pred = model.predict(images)

2024-03-18 07:13:28.241623: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8905
2024-03-18 07:13:28.310158: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


27/27 [==============================] - 1s 11ms/step


In [14]:
n_classes = 4

y_pred = get_labels(pred)
total = len(y_pred)

correct = 0
for i in range(len(y_pred)):
    if y_pred[i] == labels[i]:
        correct += 1

accuracy = correct / total

print(correct)
print(total)

# Initialize variables to store weighted sums
weighted_sum_sensitivity = 0
weighted_sum_specificity = 0
weighted_sum_precision = 0
weighted_sum_accuracy = 0
weighted_sum_f1 = 0

class_metrics = []

for i in range(n_classes):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for j in range(len(y_pred)):
        if labels[j] == i and y_pred[j] == i:
            TP += 1
        elif labels[j] != i and y_pred[j] == i:
            FP += 1
        elif labels[j] == i and y_pred[j] != i:
            FN += 1
        elif labels[j] != i and y_pred[j] != i:
            TN += 1


    sensitivity = TP / (TP + FN) if (TP + FN) > 0 else -1
    specificity = TN / (TN + FP) if (TN + FP) > 0 else -1
    precision = TP / (TP + FP) if (TP + FP) > 0 else -1
    class_accuracy = (TP + TN) / (TP + FN + TN + FP)
    recall = sensitivity
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else -1

    class_metrics.append({
        'class': int(i),
        'TP': TP,
        'FP': FP,
        'TN': TN,
        'FN': FN,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'Precision': precision,
        'Accuracy': class_accuracy,
        'F1 Score': f1,
        'Relative Frequency': relative_freq[i]
    })

    # Calculate weighted sums
    weighted_sum_sensitivity += sensitivity * relative_freq[i]
    weighted_sum_specificity += specificity * relative_freq[i]
    weighted_sum_precision += precision * relative_freq[i]
    weighted_sum_accuracy += class_accuracy * relative_freq[i]
    weighted_sum_f1 += f1 * relative_freq[i]



print(accuracy)


784
840
0.9333333333333333


In [15]:
import pandas as pd

def create_metrics_dataframe(class_metrics):
    # Create a dictionary to store the metrics for each class
    metrics_dict = {
        'Class': [metrics['class'] for metrics in class_metrics],
        'Specificity': [metrics['Specificity'] for metrics in class_metrics],
        'Sensitivity': [metrics['Sensitivity'] for metrics in class_metrics],
        'Precision': [metrics['Precision'] for metrics in class_metrics],
        'Accuracy': [metrics['Accuracy'] for metrics in class_metrics],
        'F1 Score': [metrics['F1 Score'] for metrics in class_metrics]
    }

    # Create a Pandas DataFrame from the metrics dictionary
    metrics_df = pd.DataFrame(metrics_dict)

    return metrics_df

# Usage:
# Assuming you have class_metrics list defined as in your previous code
metrics_dataframe = create_metrics_dataframe(class_metrics)

# Calculate weighted average metrics
weighted_avg_metrics = {
    'Class': 'Weighted Avg',
    'Specificity': weighted_sum_specificity,  # Replace with your actual values
    'Sensitivity': weighted_sum_sensitivity,  # Replace with your actual values
    'Precision': weighted_sum_precision,      # Replace with your actual values
    'Accuracy': weighted_sum_accuracy,        # Replace with your actual values
    'F1 Score': weighted_sum_f1               # Replace with your actual values
}


# Convert the weighted average metrics dictionary to a DataFrame for easier concatenation
weighted_avg_metrics_df = pd.DataFrame([weighted_avg_metrics])

# Concatenate the original dataframe with the new dataframe containing weighted average metrics
metrics_dataframe = pd.concat([metrics_dataframe, weighted_avg_metrics_df], ignore_index=True)
metrics_dataframe = metrics_dataframe.round(3)
print(metrics_dataframe)

metrics_dataframe.to_csv('classwise_metrics.csv', index=False)

          Class  Specificity  Sensitivity  Precision  Accuracy  F1 Score
0             0        0.951        0.922      0.876     0.943     0.898
1             1        0.974        0.843      0.836     0.956     0.840
2             2        0.998        0.979      0.996     0.993     0.987
3             3        0.990        0.943      0.976     0.975     0.959
4  Weighted Avg        0.979        0.933      0.935     0.969     0.934


In [16]:
for i in range(len(y_pred)):
  print(y_pred[i], labels[i])

0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
1 0.0
1 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
1 0.0
0 0.0
0 0.0
0 0.0
1 0.0
0 0.0
0 0.0
1 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
1 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
1 0.0
0 0.0
0 0.0
0 0.0
0 0.0
3 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
1 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
1 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
1 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
1 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
2 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.